In [1]:
#r "nuget: Microsoft.DotNet.Cli.Utils"

open Microsoft.DotNet.Cli.Utils

let runDotNetBuild() =
    let result = 
        Command
            .CreateDotNet("build", [])
            .CaptureStdOut()
            .CaptureStdErr()
            .Execute()

    let output = result.StdOut
    let error = result.StdErr
    let exitCode = result.ExitCode

    (output, error, exitCode)

// Usage
let (output, error, exitCode) = runDotNetBuild()

printfn "Exit Code: %d" exitCode
printfn "Output:\n%s" output
printfn "Error:\n%s" error

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.DotNet.Cli.Utils, 2.0.0

Exit Code: 0
Output:
MSBuild version 17.8.5+b5265ef37 for .NET
  Determining projects to restore...
  All projects are up-to-date for restore.
  core -> D:\programming\stock-analysis\src\core\bin\Debug\net8.0\core.dll
  core.fs -> D:\programming\stock-analysis\src\core.fs\bin\Debug\net8.0\core.fs.dll
  secedgar -> D:\programming\stock-analysis\src\infrastructure\secedgar\bin\Debug\net8.0\secedgar.dll
  stripe -> D:\programming\stock-analysis\src\infrastructure\stripe\bin\Debug\net8.0\stripe.dll
  twilioclient -> D:\programming\stock-analysis\src\infrastructure\twilioclient\bin\Debug\net8.0\twilioclient.dll
  storage.shared -> D:\programming\stock-analysis\src\infrastructure\storage.shared\bin\Debug\net8.0\storage.shared.dll
  timezonesupport -> D:\programming\stock-analysis\src\infrastructure\timezonesupport\bin\Debug\net8.0\timezonesupport.dll
  csvparser -> D:\programming\stock-analysis\src\infrastructure\csvparser\bin\Debug\net8.0\csvparser.dll
  securityutils -> D:\programming\stoc

In [2]:
let buildDir = "./bin/Debug/net8.0" 
let copyDir = "./bin/Debug/notebook"

// Clear existing copy directory
if System.IO.Directory.Exists(copyDir) then 
    System.IO.Directory.Delete(copyDir, true)

let rec copyBuildDir (source : string) (target: string) =
    System.IO.Directory.CreateDirectory(target) |> ignore  
    for file in System.IO.Directory.GetFiles(source) do 
        File.Copy(file, System.IO.Path.Combine(target, System.IO.Path.GetFileName(file)))

    for dir in System.IO.Directory.GetDirectories(source) do
        let newDir = System.IO.Path.Combine(target, System.IO.Path.GetFileName(dir)) 
        copyBuildDir dir newDir

copyBuildDir buildDir copyDir

In [3]:
#r "./bin/Debug/notebook/studies.dll"
#r "./bin/Debug/notebook/core.fs.dll"

#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: MathNet.Numerics"

Installed Packages FSharp.Data, 6.4.0 MathNet.Numerics, 5.0.0 Plotly.NET.Interactive, 5.0.0

Loading extensions from `C:\Users\laimi\.nuget\packages\plotly.net.interactive\5.0.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [4]:
// read params from files
let resultsDirectory = @"D:\studies\breakout_study_2024\results_20240621_090304"
let studiesDirectory = @"D:\studies\breakout_study_2024"

//let resultsDirectory = System.IO.File.ReadAllText("results_directory.txt")
//let studiesDirectory = System.IO.File.ReadAllText("study_directory.txt")
let inputFilename    = $"{studiesDirectory}\\signals.csv"

System.Console.WriteLine("Study directory: " + studiesDirectory)
System.Console.WriteLine("Input filename: " + inputFilename)
System.Console.WriteLine("Results directory: " + resultsDirectory)

Study directory: D:\studies\breakout_study_2024
Input filename: D:\studies\breakout_study_2024\signals.csv
Results directory: D:\studies\breakout_study_2024\results_20240621_090304


In [5]:
open FSharp.Data
open Plotly.NET
open core.fs.Stocks

let outputToConsole (message:string) =
    System.Console.WriteLine(message)
        
// functions that deal with outcomes and turn them into summaries, describes them, etc
let columnChart label keys values =
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
let histogramChart label bins values =
    let chart = 
        Chart.Histogram(
            X = values,
            NBinsX = bins,
            Name = label
        )
    chart.Display() |> ignore

let outputSummary outputFunc includeDistributionChart (summary:studies.Trading.TradeSummary) = 

    // this ensures that the numbers align instead of negative EV taking up more space
    let evPositiveSign =
        match summary.EV with
        | x when x >= 0m -> "+"
        | _ -> ""
    
    let format = "0.00"
    let percentFormat = "00.00%"
    outputFunc $"{evPositiveSign}{summary.EV.ToString(percentFormat)} EV -> {summary.WinPct.ToString(percentFormat)} \t {summary.AvgWin.ToString(percentFormat)} \t {summary.AvgLoss.ToString(percentFormat)} \t {summary.AvgGainLoss.ToString(format)} avg gain/loss \t {summary.TotalGain.ToString(format)} total gain  \t [{summary.NumberOfTrades} trades] \t {summary.StrategyName}"
        
    match includeDistributionChart with
    | true ->
        histogramChart "Gain Distribution" 40 (summary.Gains)
    | false -> ()
    
    
// filter out outcomes that have minimum of trades we are interested in
let minimumTradesPerCategory = 10

let createTradeSummaries outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Trading.TradeOutcome) -> t.Strategy) // all strategies, no filters

    tradesGroupedByStrategy
    |> Seq.map (
        fun (strategy, trades) ->
            let name = $"{strategy}"
            let summary = trades |> studies.Trading.TradeSummary.create name
            match summary.NumberOfTrades with
            | x when x < minimumTradesPerCategory -> None
            | _ -> Some summary
    )
    |> Seq.choose id
    

type OutcomeGrouping =
    | Signal
    | Strategy
    
let describeOutcomes outputFileOption includeEVChart includeDistributionChart tradeSummaries =
    
    let outputContent message =
        match outputFileOption with
        | None -> ()
        | Some outputFile -> System.IO.File.AppendAllText(outputFile, message + System.Environment.NewLine)
        System.Console.WriteLine(message)
        
    let groupedData = tradeSummaries |> Seq.groupBy (fun (o:studies.Trading.TradeSummary) -> o.StrategyName)
    
    groupedData
    |> Seq.iter (
        fun (groupingName,summaries) ->
            
            outputContent $"{groupingName}"
            
            let sortedSummaries = summaries |> Seq.sortByDescending (fun summary -> summary.EV)
            
            sortedSummaries |> Seq.iter (fun summary -> outputSummary outputContent includeDistributionChart summary)
            
            if includeEVChart then
                let keys = sortedSummaries |> Seq.map (fun summary -> summary.StrategyName)
                let values = sortedSummaries |> Seq.map (fun summary -> summary.EV)
            
                columnChart groupingName keys values
                
            outputContent ""
            outputContent ""
    )
    
let highlightStrategies tradeSummaries =
        
    let sortFunctions = [
        ("EV", fun (t:studies.Trading.TradeSummary) -> t.EV)
        //("Total Gain", fun (t:studies.ScreenerStudy.TradeSummary) -> t.TotalGain)
        //("Win %", fun (t:studies.ScreenerStudy.TradeSummary) -> t.WinPct)
        //("Avg Gain", fun (t:studies.ScreenerStudy.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Strategies sorted by: {sortLabel}"
        
        let summaryFunc = outputSummary outputToConsole false
        let descending =
            tradeSummaries
            |> Seq.sortByDescending sortFunction
            |> Seq.iter summaryFunc
            
        printfn ""
        printfn ""
    )
    
let saveTradeOutcomesToCsv resultsDirectory tradeSummaries =
    tradeSummaries
    |> Seq.iter(fun (summary:studies.Trading.TradeSummary) ->
        let invalidCharacters = System.IO.Path.GetInvalidFileNameChars()
        let cleanedFilename = invalidCharacters |> Seq.fold (fun (n:string) (c:char) -> n.Replace(c, '_')) summary.StrategyName
        let finalFilename = $"{cleanedFilename}.csv"
        let rows = studies.BreakoutStudy.createTradeOutcomesOutput summary.Trades
        let finalPath = System.IO.Path.Combine(resultsDirectory,finalFilename)
        rows.Save(finalPath)
    )        

In [6]:
let inputSignals =
    inputFilename
    |> studies.BreakoutStudy.BreakoutSignalRecord.Load
    |> _.Rows
    |> Seq.map studies.BreakoutStudy.BreakoutSignalRecordWrapper
    |> List.ofSeq
    
outputToConsole $"Loaded {inputSignals.Length} signals"


Loaded 33518 signals


In [7]:
open MathNet.Numerics

// analyze data first
// visualize current slopes
let createHistogram label buckets (func:studies.BreakoutStudy.BreakoutSignalRecordWrapper->decimal) (filter:decimal->bool) (records:studies.BreakoutStudy.BreakoutSignalRecordWrapper list) =
    let slopes = records |> List.map func |> List.filter filter
    
    histogramChart label buckets slopes
    
let showStatistics label (func:studies.BreakoutStudy.BreakoutSignalRecordWrapper->float) (records:studies.BreakoutStudy.BreakoutSignalRecordWrapper list) =
    let values = records |> List.map func
    let stats = MathNet.Numerics.Statistics.DescriptiveStatistics(values)
    let mean = stats.Mean
    let stdDev = stats.StandardDeviation
    $"{label}: count: {records.Length}, mean: {mean}, stdDev: {stdDev}" |> outputToConsole
    stats
    
  
let priceSlopeStats = showStatistics "Price Slope" (fun s -> s.Row.PriceSlope |> float) inputSignals
createHistogram "Price Slope" 1000 (fun s -> s.Row.PriceSlope) (fun slope -> slope > -10m && slope < 10m) inputSignals

let priceAngleStats = showStatistics "Price Angle" (fun s -> s.Row.PriceDegrees |> float) inputSignals
createHistogram "Price Degrees" 10 (fun s -> s.Row.PriceDegrees) (fun s -> true) inputSignals

let volumeSlopeStats = showStatistics "Volume Slope" (fun s -> s.Row.VolumeSlope |> float) inputSignals
createHistogram "Volume Slope" 1000 (fun s -> s.Row.VolumeSlope) (fun volume -> true) inputSignals

let volumeAngleStats = showStatistics "Volume Angle" (fun s -> s.Row.VolumeDegrees |> float) inputSignals
createHistogram "Volume Degrees" 10 (fun s -> s.Row.VolumeDegrees) (fun s -> true) inputSignals

let volumeRateStats = showStatistics "Volume Rate" (fun s -> s.Row.BreakoutVolumeRate |> float) inputSignals
createHistogram "Volume Rate" 100 (fun s -> s.Row.BreakoutVolumeRate) (fun v -> true) inputSignals

// let signalsOutsidePriceSlope = inputSignals |> List.filter (fun s -> s.Row.PriceSlope > decimal (priceSlopeStats.Mean + priceSlopeStats.StandardDeviation * 1.5) * 1m)
// signalsOutsidePriceSlope |> List.sortBy (fun s -> s.Row.PriceSlope) |> List.iter(fun s -> $"{s.Row.Ticker} - {s.Row.Date} - {s.Row.PriceSlope}" |> outputToConsole)

Price Slope: count: 33518, mean: 0.0011054730540344673, stdDev: 1.544550753297985


<!-- Plotly chart will be drawn inside this DIV -->

Price Angle: count: 33518, mean: 1.5346406749101713, stdDev: 29.086326217171298


<!-- Plotly chart will be drawn inside this DIV -->

Volume Slope: count: 33518, mean: -13405.125543943468, stdDev: 46737.33815744923


<!-- Plotly chart will be drawn inside this DIV -->

Volume Angle: count: 33518, mean: -8.869522972239166, stdDev: 6.427647124762174


<!-- Plotly chart will be drawn inside this DIV -->

Volume Rate: count: 33518, mean: 2.305791255387769, stdDev: 14.259962093002754


<!-- Plotly chart will be drawn inside this DIV -->

In [8]:

// this step is for any additional filtering, but right now not doing
// any filtering, and only casting to ISignal that Trading module understands
let signals = 
    inputSignals
    |> Seq.cast<studies.DataHelpers.ISignal>
    |> Seq.toList

outputToConsole $"After filtering, we have {signals.Length} signals"

let priceFunc ticker = studies.DataHelpers.getPricesFromCsv studiesDirectory ticker

let verbosityOn = false

let strategies = [
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None None   // no stop, buy and hold or sell and hold
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 5) None // no stop, but only hold for 5 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 5) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 10) None // no stop, but only hold for 10 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 10) None    
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 30) None // no stop, but only hold for 30 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 30) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 60) None // no stop, but only hold for 60 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 60) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None (Some 0.05m) // 5% stop
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None (Some 0.1m) // 10% stop
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None (Some 0.1m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None (Some 0.2m) // 20% stop
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None (Some 0.2m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 30) (Some 0.05m) // 5% stop plus holding at most 30 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 30) (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 60) (Some 0.05m) // 5% stop plus holding at most 60 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 60) (Some 0.05m)
    studies.Trading.strategyWithSignalOpenAsStop verbosityOn  // signal open as stop
    studies.Trading.strategyWithSignalCloseAsStop verbosityOn // signal close as stop
    studies.Trading.strategyWithTrailingStop verbosityOn StockPositionType.Long 0.1m  // trailing stop of 10%
    studies.Trading.strategyWithTrailingStop verbosityOn StockPositionType.Short 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Long 0.05m 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Short 0.05m 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Long 0.1m 0.2m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Short 0.1m 0.2m
]

let outcomes = 
    strategies
    |> studies.Trading.runTrades priceFunc signals
    |> Async.RunSynchronously
    |> Seq.toList

printfn "Finished running trades, generated:"
printfn $"{outcomes.Length} trade outcomes"

After filtering, we have 33518 signals
Records: 33518, dates: 755, tickers: 3269, screenerIds: 1
Minimum date: "2021-05-20"
Maximum date: "2024-05-20"

Ensured that data has prices
Records: 33518, dates: 755, tickers: 3269, screenerIds: 1
Minimum date: "2021-05-20"
Maximum date: "2024-05-20"

Executing trades...
  Executing strategy 1...
  Executing strategy 2...
  Executing strategy 3...
  Executing strategy 4...
  Executing strategy 5...
  Executing strategy 6...
  Executing strategy 7...
  Executing strategy 8...
  Executing strategy 9...
  Executing strategy 10...
  Executing strategy 11...
  Executing strategy 12...
  Executing strategy 13...
  Executing strategy 14...
  Executing strategy 15...
  Executing strategy 16...
  Executing strategy 17...
  Executing strategy 18...
  Executing strategy 19...
  Executing strategy 20...
  Executing strategy 21...
  Executing strategy 22...
  Executing strategy 23...
  Executing strategy 24...
  Executing strategy 25...
  Executing strategy

In [9]:
// leave this cell for outcome checking from time to time when troubleshooting
//outcomes |> List.filter (fun (o:studies.ScreenerStudy.TradeOutcomeOutput.Row) -> o.Screenerid = 28 && o.OpenPrice < o.Sma200 && o.Strategy = "Buy")

In [10]:
printfn "Generating trade summaries..."
let tradeSummaries = outcomes |> createTradeSummaries |> Seq.toList
printfn $"{tradeSummaries.Length} summaries generated"

printfn "Saving trade summaries..."
tradeSummaries |> saveTradeOutcomesToCsv resultsDirectory

Generating trade summaries...
28 summaries generated
Saving trade summaries...


In [11]:
"Highlighting trades" |> outputToConsole

tradeSummaries |> highlightStrategies

Highlighting trades
Strategies sorted by: EV
+14.14% EV -> 53.47% 	 62.81% 	 -41.79% 	 1.50 avg gain/loss 	 462.29 total gain  	 [3269 trades] 	 Buy
+04.86% EV -> 32.39% 	 60.33% 	 -21.73% 	 2.78 avg gain/loss 	 345.26 total gain  	 [7097 trades] 	 Buy SL of 0.20%
+02.68% EV -> 21.45% 	 58.48% 	 -12.55% 	 4.66 avg gain/loss 	 290.86 total gain  	 [10824 trades] 	 Buy SL of 0.10%
+01.89% EV -> 14.49% 	 56.45% 	 -07.36% 	 7.67 avg gain/loss 	 283.97 total gain  	 [15000 trades] 	 Buy SL of 0.05%
+00.97% EV -> 12.99% 	 42.31% 	 -05.20% 	 8.14 avg gain/loss 	 186.68 total gain  	 [19036 trades] 	 Buy and use signal open as stop
+00.89% EV -> 51.00% 	 16.41% 	 -15.26% 	 1.08 avg gain/loss 	 155.35 total gain  	 [17198 trades] 	 Buy hold for 60 bars
+00.68% EV -> 39.28% 	 20.37% 	 -12.06% 	 1.69 avg gain/loss 	 133.29 total gain  	 [19602 trades] 	 Buy SL of 0.10% PT of 0.20%
+00.58% EV -> 17.31% 	 16.32% 	 -02.71% 	 6.01 avg gain/loss 	 151.65 total gain  	 [25223 trades] 	 Buy and use sign

In [12]:
// tradeSummaries
// |> describeOutcomes None false false
//
// printfn "Finished"

In [13]:
let exploreCertainStrategies() =
    let strategiesOfInterest = [
        "SL of"
        "as stop"
    ]

    let summarizeSubset (name:string) =
        tradeSummaries
            |> Seq.filter (fun summary -> summary.StrategyName.Contains(name))
            |> Seq.sortByDescending (fun summary -> summary.EV)
            |> Seq.iter (fun summary -> 
                outputSummary outputToConsole false summary
            )

        printfn ""

    strategiesOfInterest |> List.iter (fun n -> summarizeSubset n)

// exploreCertainStrategies()

In [14]:
// sanity check, print random trades, look over them, investigate specific trades if needed
let positionDescription (signal:studies.Trading.TradeOutcome) =
    // it will consist of components that describe signal
    // gap up or gap down
    // price>20, price<20, put together into a string

    let signalDescription =
        match signal.Signal.Screenerid |> Option.defaultValue 0 with
        | 28 -> "New High"
        | 29 -> "Top Gainer"
        | 30 -> "Top Loser"
        | 31 -> "New Low"
        | _ -> "Generic Signal"
    
    String.concat " " [signalDescription]
        
let outcomeDescription (x:studies.Trading.TradeOutcome) =
    let gain = x.PercentGain.ToString("##.##%")
    let signal = x |> positionDescription
    printfn $"{x.Ticker} \t {x.Strategy} \t Profit: {gain} \t  {x.Opened:d} -> {x.Closed:d} \t {x.OpenPrice} -> {x.ClosePrice}. VA: {signal}, strat: {x.Strategy}"
    
let random = System.Random()

let tradeStrategyOfInterest = "Buy"

//let filterFunc x = x |> signalDescription |> _.Contains(tradeStrategyOfInterest)
let filterFunc (x:studies.Trading.TradeOutcome) = x.Strategy = tradeStrategyOfInterest

outcomes
|> Seq.filter filterFunc
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription

printfn ""
printfn $"Max gain trade from {tradeStrategyOfInterest}"

outcomes
|> Seq.filter filterFunc
|> Seq.sortByDescending (fun x -> x.PercentGain)
|> Seq.truncate 10
|> Seq.iter outcomeDescription

printfn ""
printfn "Max gain overall"

outcomes
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

KARO 	 Buy 	 Profit: -18.78% 	  2021-07-30 -> 2024-05-21 	 35.9267 -> 29.18. VA: Generic Signal, strat: Buy
UFI 	 Buy 	 Profit: -73.18% 	  2021-09-01 -> 2024-05-21 	 23.45 -> 6.29. VA: Generic Signal, strat: Buy
MANH 	 Buy 	 Profit: 103.32% 	  2022-06-16 -> 2024-05-21 	 110.99 -> 225.67. VA: Generic Signal, strat: Buy
TWIN 	 Buy 	 Profit: 11.02% 	  2021-10-29 -> 2024-05-21 	 12.8 -> 14.21. VA: Generic Signal, strat: Buy
IDR 	 Buy 	 Profit: 9.64% 	  2024-04-23 -> 2024-05-21 	 9.54 -> 10.46. VA: Generic Signal, strat: Buy
SEED 	 Buy 	 Profit: -38.87% 	  2022-07-22 -> 2024-05-21 	 9.75 -> 5.96. VA: Generic Signal, strat: Buy
NUWE 	 Buy 	 Profit: -99.7% 	  2022-03-09 -> 2024-05-21 	 90.3 -> 0.269. VA: Generic Signal, strat: Buy
VSCO 	 Buy 	 Profit: -57.22% 	  2021-12-20 -> 2024-05-21 	 48.08 -> 20.57. VA: Generic Signal, strat: Buy
BFH 	 Buy 	 Profit: -4.71% 	  2022-06-15 -> 2024-05-21 	 44.63 -> 42.53. VA: Generic Signal, strat: Buy
EWTX 	 Buy 	 Profit: 86.16% 	  2023-02-27 -> 2024-05-21 

In [15]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("EV", fun (s:studies.Trading.TradeSummary) -> s.EV)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("Win %", fun s -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Number of Trades", fun s -> s.NumberOfTrades)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    outputToConsole $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    outputToConsole $"mapped keys and values for {label}, charting..."
    
    // values |> List.zip keys |> List.iter (fun (l,v) -> outputToConsole $"{l}: {v}")
    
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
    outputToConsole ""
)

sorting for EV...
mapped keys and values for EV, charting...


<!-- Plotly chart will be drawn inside this DIV -->


sorting for Win/Loss...
mapped keys and values for Win/Loss, charting...


<!-- Plotly chart will be drawn inside this DIV -->


sorting for Win %...
mapped keys and values for Win %, charting...


...


sorting for Avg Loss...
mapped keys and values for Avg Loss, charting...


<!-- Plotly chart will be drawn inside this DIV -->


sorting for Number of Trades...
mapped keys and values for Number of Trades, charting...


<!-- Plotly chart will be drawn inside this DIV -->


input.fsx (9,35)-(9,51) typecheck warning This expression uses the implicit conversion 'Decimal.op_Implicit(value: int) : decimal' to convert type 'int' to type 'decimal'. See https://aka.ms/fsharp-implicit-convs. This warning may be disabled using '#nowarn "3391".



In [16]:
// let's take only the trades where EV is positive
outputToConsole "Positive EVs only, sorted by EV descending"

tradeSummaries
|> List.filter (fun s -> s.EV > 0m)
|> List.sortByDescending (fun s -> s.EV)
|> List.iter (fun s -> outputSummary outputToConsole false s)

Positive EVs only, sorted by EV descending
+14.14% EV -> 53.47% 	 62.81% 	 -41.79% 	 1.50 avg gain/loss 	 462.29 total gain  	 [3269 trades] 	 Buy
+04.86% EV -> 32.39% 	 60.33% 	 -21.73% 	 2.78 avg gain/loss 	 345.26 total gain  	 [7097 trades] 	 Buy SL of 0.20%
+02.68% EV -> 21.45% 	 58.48% 	 -12.55% 	 4.66 avg gain/loss 	 290.86 total gain  	 [10824 trades] 	 Buy SL of 0.10%
+01.89% EV -> 14.49% 	 56.45% 	 -07.36% 	 7.67 avg gain/loss 	 283.97 total gain  	 [15000 trades] 	 Buy SL of 0.05%
+00.97% EV -> 12.99% 	 42.31% 	 -05.20% 	 8.14 avg gain/loss 	 186.68 total gain  	 [19036 trades] 	 Buy and use signal open as stop
+00.89% EV -> 51.00% 	 16.41% 	 -15.26% 	 1.08 avg gain/loss 	 155.35 total gain  	 [17198 trades] 	 Buy hold for 60 bars
+00.68% EV -> 39.28% 	 20.37% 	 -12.06% 	 1.69 avg gain/loss 	 133.29 total gain  	 [19602 trades] 	 Buy SL of 0.10% PT of 0.20%
+00.58% EV -> 17.31% 	 16.32% 	 -02.71% 	 6.01 avg gain/loss 	 151.65 total gain  	 [25223 trades] 	 Buy and use signal

In [17]:
let benchmarkStrategyName = "Sell"

//tradeSummaries |> List.map _.StrategyName |> List.distinct |> List.iter outputToConsole
let benchmarkStrategy = tradeSummaries |> List.find (fun s -> s.StrategyName = benchmarkStrategyName)

outputSummary outputToConsole false benchmarkStrategy

-14.14% EV -> 46.53% 	 41.79% 	 -62.81% 	 0.67 avg gain/loss 	 -462.29 total gain  	 [3269 trades] 	 Sell


In [18]:
//let newHighsWithPriceBelow20 = outcomes |> Seq.filter (fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> Option.defaultValue 0 = 28 && o.OpenPrice < (o.Sma20 |> Option.defaultValue 0m) )

//let count = newHighsWithPriceBelow20 |> Seq.length

//printfn "Found %i" count
//newHighsWithPriceBelow20 |> Seq.take 10 |> Seq.iter outcomeDescription

In [19]:
//let topLoserPriceAbove20 = outcomes |> Seq.filter (fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> Option.defaultValue 0 = 30 && o.OpenPrice > (o.Sma20 |> Option.defaultValue 0m) && o.Strategy = "Sell")

//let count = topLoserPriceAbove20 |> Seq.length

//printfn "Found %i" count
//topLoserPriceAbove20 |> Seq.take 30 |> Seq.iter outcomeDescription

In [45]:
type InternalOutcomeSummary = {
    ev: decimal
    win_pct: decimal
    avg_win: decimal
    avg_loss: decimal
    numberOfTrades: int
    label: string
    outcomes: studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row seq
}
    with
        member this.Describe() =
            $"EV : {this.ev:P}\t Win: {this.win_pct:P}, Avg Win: {this.avg_win:P}, Avg Loss: {this.avg_loss:P}, total: {this.numberOfTrades}, {this.label}"


In [53]:

let summarizeRecords label (outcomes:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row seq) = 
    let numberOfTrades = outcomes |> Seq.length
    
    if numberOfTrades = 0 then
        {
            ev = 0m
            win_pct = 0m
            avg_win = 0m
            avg_loss = 0m
            numberOfTrades = 0
            label = label
            outcomes = []
        }
    else
        let winners = outcomes |> Seq.filter (fun o -> o.PercentGain > 0m)
        let losers = outcomes |> Seq.filter (fun o -> o.PercentGain < 0m)
        let numberOfWinners = winners |> Seq.length
        let numberOfLosers = losers |> Seq.length
        let win_pct = decimal numberOfWinners / decimal numberOfTrades
        let avg_win =
            match numberOfWinners with
            | 0 -> 0m
            | _ -> winners |> Seq.averageBy (_.PercentGain)
        let avg_loss =
            match numberOfLosers with
            | 0 -> 0m
            | _ -> losers |> Seq.averageBy (_.PercentGain)
        let avg_gain_loss =
            match avg_loss with
            | 0m -> 0m
            | _ -> avg_win / avg_loss |> Math.Abs
        let ev = win_pct * avg_win - (1m - win_pct) * (avg_loss |> Math.Abs)
        let totalGain = outcomes |> Seq.sumBy (_.PercentGain)

        {
            ev = ev
            win_pct = win_pct
            avg_win = avg_win
            avg_loss = avg_loss
            numberOfTrades = numberOfTrades
            label = label
            outcomes = outcomes
        }

In [57]:
let load filename =
    let filepath = @$"{resultsDirectory}\{filename}"
    studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Load(filepath)
    
let summarize (dataSetName:string) (records:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row seq) =

    [
        summarizeRecords $"{dataSetName} All" records

        // let's take only those that have volume rate above 
        records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 1m && r.BreakoutVolumeRate < 2m) |> summarizeRecords $"{dataSetName} Breakout 1-2"
        records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 2m && r.BreakoutVolumeRate < 3m) |> summarizeRecords $"{dataSetName} Breakout 2-3"
        records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 3m && r.BreakoutVolumeRate < 4m) |> summarizeRecords $"{dataSetName} Breakout 3-4"
        records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 4m && r.BreakoutVolumeRate < 5m) |> summarizeRecords $"{dataSetName} Breakout 4-5"
        records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 5m) |> summarizeRecords $"{dataSetName} Breakout 5+"
        records |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 4m && r.PriceDegrees >= 20m) |> summarizeRecords $"{dataSetName} Breakout 4x+ & Price Angle <= -20 degrees"

        // let's see what's the difference between rising and falling price slope
        records |> Seq.filter (fun r -> r.PriceSlope < 0m) |> summarizeRecords $"{dataSetName} Negative Price Slope"
        records |> Seq.filter (fun r -> r.PriceSlope > 0m) |> summarizeRecords $"{dataSetName} Positive Price Slope"
    
        // let's see what the angle breakouts look like
        //records |> Seq.filter (fun r -> r.PriceDegrees <= -40m) |> summarizeRecords "Price deg < -40"
        //records |> Seq.filter (fun r -> r.PriceDegrees < -20m && r.PriceDegrees > -40m) |> summarizeRecords "Price deg -40 to -20"
        //records |> Seq.filter (fun r -> r.PriceDegrees < 20m  && r.PriceDegrees >= -20m) |> summarizeRecords "Price deg -20 to 20"
        //records |> Seq.filter (fun r -> r.PriceDegrees < 40m && r.PriceDegrees >= 20m) |> summarizeRecords "Price deg 20 to 40"
        //records |> Seq.filter (fun r -> r.PriceDegrees >= 40m) |> summarizeRecords "Price deg > 40"
    ]
        

In [99]:
let nofilter rows = rows
let above30PriceFilter rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.OpenPrice >= 30m) rows
let industryCycleUp rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.IndustryCycle = "Up") rows
let industryCycleDown rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.IndustryCycle = "Down") rows
let spyShortTermUp rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.SpyShortTermCycle = "Up") rows
let spyShortTermDown rows = Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.SpyShortTermCycle = "Down") rows
let spyLongTermUp rows =  Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.SpyLongTermCycle = "Up") rows
let spyLongTermDown rows =  Seq.filter (fun (r:studies.BreakoutStudy.BreakoutTradeOutcomeRecord.Row) -> r.SpyLongTermCycle = "Down") rows

let filters = [
    "None", nofilter
    "Price above 30", above30PriceFilter
    "Industry Cycle Up", industryCycleUp
    "Industry Cycle Down", industryCycleDown
    "Spy Short Term Up", spyShortTermUp
    "Spy Short Term Down", spyShortTermDown
    "Spy Long Term Up", spyLongTermUp
    "Spy Long Term Down", spyLongTermDown    
]

let results =
    filters
    |> List.collect(fun (filterName, filterFunc) ->

        $"************ Filter applied: {filterName}" |> outputToConsole

        tradeSummaries
        |> List.map (fun s -> s.StrategyName)
        |> List.collect (fun strategyName ->
            let csv = load $"{strategyName}.csv"
            csv.Rows |> filterFunc |> summarize $"{filterName}: {strategyName}" 
        )
    )

************ Filter applied: None
************ Filter applied: Price above 30
************ Filter applied: Industry Cycle Up
************ Filter applied: Industry Cycle Down
************ Filter applied: Spy Short Term Up
************ Filter applied: Spy Short Term Down
************ Filter applied: Spy Long Term Up
************ Filter applied: Spy Long Term Down


In [105]:
// sort by ev and print the heck out of it
results
|> List.sortByDescending (fun ir -> ir.ev)
|> List.iter( fun ir -> 
    System.Console.WriteLine(ir.Describe())
)

EV : 41.29%	 Win: 75.16%, Avg Win: 68.77%, Avg Loss: -41.89%, total: 153, Industry Cycle Down: Buy Breakout 2-3
EV : 37.23%	 Win: 66.15%, Avg Win: 72.37%, Avg Loss: -31.47%, total: 195, Spy Long Term Down: Buy Breakout 2-3
EV : 37.17%	 Win: 66.93%, Avg Win: 72.53%, Avg Loss: -34.38%, total: 626, Spy Long Term Down: Buy Negative Price Slope
EV : 33.74%	 Win: 65.56%, Avg Win: 68.48%, Avg Loss: -32.37%, total: 1141, Spy Long Term Down: Buy All
EV : 33.40%	 Win: 69.29%, Avg Win: 62.97%, Avg Loss: -33.34%, total: 482, Industry Cycle Down: Buy Negative Price Slope
EV : 29.56%	 Win: 63.88%, Avg Win: 63.32%, Avg Loss: -30.14%, total: 515, Spy Long Term Down: Buy Positive Price Slope
EV : 29.42%	 Win: 67.74%, Avg Win: 58.87%, Avg Loss: -32.42%, total: 806, Industry Cycle Down: Buy All
EV : 29.32%	 Win: 64.76%, Avg Win: 63.36%, Avg Loss: -33.26%, total: 403, Industry Cycle Up: Buy Negative Price Slope
EV : 28.67%	 Win: 62.69%, Avg Win: 68.16%, Avg Loss: -37.68%, total: 193, Spy Short Term Up: Bu

In [93]:
// sort by ev and print the heck out of it
let sellOutcomes = load "Sell.csv"

let tradesOfInterest = sellOutcomes.Rows |> Seq.filter (fun r -> r.BreakoutVolumeRate >= 5m)

let sumarryOf5XSell = summarizeRecords "5X Sell" tradesOfInterest

sumarryOf5XSell.Describe()

sumarryOf5XSell.outcomes
|> Seq.iter(fun o ->
    let date = System.DateTime.Parse(o.Date)
    let dateMinus30 = date.AddDays(-30)
    let newDate = dateMinus30.ToString("yyyy-MM-dd")
    $"{o.Ticker}, {o.Date} https://localhost:44406/stocks/{o.Ticker}/analysis?startDate={newDate}" |> outputToConsole
)

ASTL, 2021-05-25 https://localhost:44406/stocks/ASTL/analysis?startDate=2021-04-25
CONN, 2021-06-03 https://localhost:44406/stocks/CONN/analysis?startDate=2021-05-04
IRMD, 2021-06-03 https://localhost:44406/stocks/IRMD/analysis?startDate=2021-05-04
BBAI, 2021-06-04 https://localhost:44406/stocks/BBAI/analysis?startDate=2021-05-05
NAPA, 2021-06-08 https://localhost:44406/stocks/NAPA/analysis?startDate=2021-05-09
PNRG, 2021-06-25 https://localhost:44406/stocks/PNRG/analysis?startDate=2021-05-26
VEL, 2021-06-25 https://localhost:44406/stocks/VEL/analysis?startDate=2021-05-26
FRGE, 2021-06-28 https://localhost:44406/stocks/FRGE/analysis?startDate=2021-05-29
PMTS, 2021-06-30 https://localhost:44406/stocks/PMTS/analysis?startDate=2021-05-31
AADI, 2021-07-08 https://localhost:44406/stocks/AADI/analysis?startDate=2021-06-08
TBLT, 2021-07-08 https://localhost:44406/stocks/TBLT/analysis?startDate=2021-06-08
DLA, 2021-07-15 https://localhost:44406/stocks/DLA/analysis?startDate=2021-06-15
IPWR, 20

In [77]:
#r "nuget: Microsoft.DotNet.Interactive.Formatting"
open Microsoft.DotNet.Interactive.Formatting

// Enable HTML output
Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/html")

// Function to create a clickable link
let createLink (text: string) (url: string) =
    sprintf "<a href='%s' target='_blank'>%s</a>" url text

// Generate some links
let links = [
    createLink "F# Documentation" "https://docs.microsoft.com/en-us/dotnet/fsharp/"
    createLink "F# for Fun and Profit" "https://fsharpforfunandprofit.com/"
    createLink "F# Software Foundation" "https://fsharp.org/"
]

// Display the links
links |> String.concat "<br/>" |> display

Installed Packages microsoft.dotnet.interactive.formatting, 1.0.0-beta.24229.4

<a href='https://docs.microsoft.com/en-us/dotnet/fsharp/' target='_blank'>F# Documentation</a><br/><a href='https://fsharpforfunandprofit.com/' target='_blank'>F# for Fun and Profit</a><br/><a href='https://fsharp.org/' target='_blank'>F# Software Foundation</a>

In [136]:
#r "nuget: XPlot.Plotly"

open XPlot.Plotly

let csv = load "Buy.csv"

let rowsToUse = 
    csv.Rows
    |> above30PriceFilter
    |> Seq.filter(fun r -> r.BreakoutVolumeRate >= 4m)

let xLabel = "Breakout Vol Rate"
let x = rowsToUse |> Seq.map (fun s -> s.BreakoutVolumeRate)
let gains = rowsToUse |> Seq.map (fun s -> s.PercentGain * 100m)

// Create a scatter plot
let trace =
    Scatter(
        x = x,
        y = gains,
        mode = "markers",
        marker = Marker(size = 10)
    )

// Set up the layout
let layout =
    Layout(
        title = $"{xLabel} vs Gains",
        xaxis = Xaxis(title = xLabel),
        yaxis = Yaxis(title = "Gains")
    )

// Create and display the chart
[trace]
|> Chart.Plot
|> Chart.WithLayout layout
|> Chart.Show

// now histogram of gains
let histogramTrace =
    Histogram(
        x = gains,
        name = "Gains Distribution",
        marker = Marker(color = "rgba(255, 100, 102, 0.7)")
    )

[histogramTrace]
|> Chart.Plot
|> Chart.Show

Installed Packages XPlot.Plotly, 4.1.0